<a href="https://colab.research.google.com/github/YousefAbua/Intro-To-ML/blob/main/Homework2/PreProcessing_Q2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW2_Question2 (Preprocessing: Standardization)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from IPython.display import display
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import and read file
file_path = '/content/drive/My Drive/Intro to ML/Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))

# Checking Null values
housing.isnull().sum()*100/housing.shape[0]

# Paramters that are strings
varlist = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea']

# Parameters for part Q1_a and Q1_b
varlist1 = ['area','bedrooms','bathrooms','stories','parking']
varlist2 = ['area','bedrooms','bathrooms','stories','mainroad','guestroom','basement','hotwaterheating','airconditioning','parking','prefarea']

# Binary map to convery strings to numerical values in chart
def binary_map(x):
  return x.map({'yes' : 1, 'no' : 0})

# Applying binary map
housing[varlist] = housing[varlist].apply(binary_map)

# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

housing[num_vars] = preprocessing.Normalizer().fit_transform(housing[num_vars])
print(housing)

        price      area  bedrooms  bathrooms   stories  mainroad  guestroom  \
0    13300000  1.000000  0.000539   0.000270  0.000404         1          0   
1    12250000  1.000000  0.000446   0.000446  0.000446         1          0   
2    12250000  1.000000  0.000301   0.000201  0.000201         1          0   
3    12215000  1.000000  0.000533   0.000267  0.000267         1          0   
4    11410000  1.000000  0.000539   0.000135  0.000270         1          1   
..        ...       ...       ...        ...       ...       ...        ...   
540   1820000  0.999999  0.000667   0.000333  0.000333         1          0   
541   1767150  0.999999  0.001250   0.000417  0.000417         0          0   
542   1750000  1.000000  0.000552   0.000276  0.000276         1          0   
543   1750000  0.999999  0.001031   0.000344  0.000344         0          0   
544   1750000  1.000000  0.000779   0.000260  0.000519         1          0   

     basement  hotwaterheating  airconditioning   p